In [20]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(llm=llm, return_messages=True)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message("Hi i'm kajj8808, i live in south korea", "wow that is so cool!")


In [21]:
memory.load_memory_variables({"input" : "who is kajj8808"})

{'history': [SystemMessage(content='On kajj8808: kajj8808 is Human. kajj8808 lives in South Korea.')]}

In [ ]:
add_message("kajj8808 likes umamusume", "wow that is so cool!")

In [23]:
memory.load_memory_variables({"input" : "what does kajj8808 like"})

{'history': [SystemMessage(content='On kajj8808: kajj8808 is Human. kajj8808 lives in South Korea. kajj8808 likes umamusume.')]}